In [1]:
import pandas as pd
import numpy as np
import torch
import spacy as sy
import torch.nn as nn
import tqdm
from collections import Counter

from gensim.models import Word2Vec
from gensim.models.fasttext import FastText

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import *

import xgboost as xgb

seed = 2000
np.random.seed(seed)

import tensorflow as tf
import keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, MaxPooling1D, Dropout, Flatten, Dense, Bidirectional
from tensorflow.keras.preprocessing import sequence

import warnings
warnings.filterwarnings("ignore")

nlp_en = sy.load('en_core_web_sm')
all_stopwords = nlp_en.Defaults.stop_words

num_i = 4

2023-05-31 10:25:27.817360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_subtask1_preprocessed_{}.csv'.format(num_i))
print(train_df.head())

               index                                               text  label
0     train_01_0_892  the state alleged they hacked sabata petros ch...      1
1    train_01_1_2714  chale was allegedly chased group about thirty ...      0
2   train_01_10_2619  the farmworkers strike resumed tuesday when th...      1
3  train_01_100_2680  demonstrators have filed for permit hold rally...      1
4  train_01_101_3090  footage the attack which included pregnant wom...      1


In [3]:
dev_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/dev_subtask1_preprocessed_{}.csv'.format(num_i))
dev_df.head()

dev_df_labels = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask1.csv')
labels = dev_df_labels['label'].values
del(dev_df_labels)
print(labels)
print(dev_df.head())

[1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0
 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0
 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1
 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0
 0 1 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0
 0 1 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1
 0 1 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0
 0 1 0 1 1 1 0]
               index                                               text
0    train_10_0_2136  the movement was catapulted into the headlines...
1     train_10_1_350  several thousand protesters took the streets a...
2   train_10_10_3104  the protest not just about saving medha life b...
3  train_10_100_1188 

In [4]:
def generate_pos_tags(text, min_threshold):
    doc = nlp_en(text)
    pos_tags = [(i.tag_) for i in doc]
    pos_tags = Counter(pos_tags)
    pos_tags = {x: count for x, count in pos_tags.items() if count > min_threshold}
    return pos_tags

def generate_tokens_prob_freq(tokens):
    dct={}
    for i in tokens:
        dct[i]=0
    for i in tokens:
        dct[i]+=1
    prob_freq = {key:float(value)/sum(dct.values()) for (key,value) in dct.items()}
    
    return prob_freq

train_df['pos_tags'] = train_df['text'].apply(lambda x: generate_pos_tags(x, 2))
dev_df['pos_tags'] = train_df['text'].apply(lambda x : generate_pos_tags(x, 2))
train_df['pos_tags_prob'] = train_df['pos_tags'].apply(lambda x: generate_tokens_prob_freq(x))
dev_df['pos_tags_prob'] = dev_df['pos_tags'].apply(lambda x: generate_tokens_prob_freq(x))
train_df.head()

,index,text,label,pos_tags,pos_tags_prob
0,train_01_0_892,the state alleged they hacked sabata petros ch...,1,"{'NN': 6, 'NNP': 7, 'CD': 7}","{'NN': 0.3333333333333333, 'NNP': 0.3333333333..."
1,train_01_1_2714,chale was allegedly chased group about thirty ...,0,"{'NN': 4, 'NNS': 3}","{'NN': 0.5, 'NNS': 0.5}"
2,train_01_10_2619,the farmworkers strike resumed tuesday when th...,1,{},{}
3,train_01_100_2680,demonstrators have filed for permit hold rally...,1,"{'NNS': 8, 'NN': 6, 'NNP': 4, 'VBD': 3, 'CC': 3}","{'NNS': 0.2, 'NN': 0.2, 'NNP': 0.2, 'VBD': 0.2..."
4,train_01_101_3090,footage the attack which included pregnant wom...,1,"{'NN': 8, 'VBD': 3, 'JJ': 3, 'VBG': 7, 'NNS': ...","{'NN': 0.14285714285714285, 'VBD': 0.142857142..."


In [5]:
train_df_pos_prob = pd.json_normalize(train_df['pos_tags_prob'])
dev_df_pos_prob = pd.json_normalize(dev_df['pos_tags_prob'])
train_df_pos_prob.replace(np.nan, 0, inplace=True)
dev_df_pos_prob.replace(np.nan, 0, inplace=True)
print(train_df_pos_prob.head())
print(dev_df_pos_prob.head())

         NN       NNP        CD       NNS       VBD        CC        JJ  \
0  0.333333  0.333333  0.333333  0.000000  0.000000  0.000000  0.000000   
1  0.500000  0.000000  0.000000  0.500000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.200000  0.200000  0.000000  0.200000  0.200000  0.200000  0.000000   
4  0.142857  0.000000  0.000000  0.142857  0.142857  0.142857  0.142857   

        VBG        IN   DT  ...   VB   RB  VBP  VBN  VBZ  PRP$  HYPH  PRP  \
0  0.000000  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0   
1  0.000000  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0   
2  0.000000  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0   
3  0.000000  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0   
4  0.142857  0.142857  0.0  ...  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0   

    MD  JJR  
0  0.0  0.0  
1  0.0  0.0  
2  0.0  0.0  
3  0.0  0.0  
4  0.0  0.0  

[

In [6]:
print(train_df_pos_prob.shape)
print(dev_df_pos_prob.shape)
print(train_df_pos_prob.columns)
print(dev_df_pos_prob.columns)

columns = list(set(train_df_pos_prob.columns) - set(dev_df_pos_prob.columns))
print(columns)

for col_name in columns:
    if col_name not in train_df_pos_prob.columns:
        train_df_pos_prob[col_name]=0

    if col_name not in dev_df_pos_prob.columns:
        dev_df_pos_prob[col_name]=0
        
print(train_df_pos_prob.shape)
print(dev_df_pos_prob.shape)

(3075, 21)
(340, 18)
Index(['NN', 'NNP', 'CD', 'NNS', 'VBD', 'CC', 'JJ', 'VBG', 'IN', 'DT', 'WDT',
       'VB', 'RB', 'VBP', 'VBN', 'VBZ', 'PRP$', 'HYPH', 'PRP', 'MD', 'JJR'],
      dtype='object')
Index(['NN', 'NNP', 'CD', 'NNS', 'VBD', 'CC', 'JJ', 'VBG', 'IN', 'DT', 'WDT',
       'VB', 'RB', 'VBP', 'VBN', 'VBZ', 'PRP$', 'HYPH'],
      dtype='object')
['JJR', 'PRP', 'MD']
(3075, 21)
(340, 21)


In [7]:
scaler = MinMaxScaler()
x = train_df_pos_prob.values
x = scaler.fit_transform(x)
y = train_df['label'].values

x_dev = dev_df_pos_prob.values
y_dev = labels

# Calculating Classweights
class_weights = compute_class_weight(
    class_weight = "balanced",
    classes = np.unique(y),
    y = y
)
class_weights = dict(zip(np.unique(y), class_weights))

count_0 = np.unique(y, return_counts=True)[1][0]
count_1 = np.unique(y, return_counts=True)[1][1]
estimate = count_0/count_1

cv = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

print(class_weights, estimate)

{0: 1.0596140592694694, 1: 0.9467364532019704} 0.8934729064039408


In [8]:
xgb_model = xgb.XGBClassifier(scale_pos_weight=estimate)

parameters = {
            'objective':['binary:logistic'],
            'learning_rate': [0.1, 0.01, 0.001, 0.0001], 
            'max_depth': [5, 6, 7, 8],
            'n_estimators': [1000], #number of trees, change it to 1000 for better results
            'seed': [1337]
        }

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=cv, 
                   verbose=0, refit=True)

clf.fit(x, y)
print(clf.best_params_, clf.best_score_)

y_pred = clf.predict(x_dev)
print(classification_report(y_dev, y_pred))
print(f1_score(y_dev, y_pred))
print(recall_score(y_dev, y_pred))
print(precision_score(y_dev, y_pred))
print(matthews_corrcoef(y_dev, y_pred))

{'learning_rate': 0.001, 'max_depth': 6, 'n_estimators': 1000, 'objective': 'binary:logistic', 'seed': 1337} 0.6074796747967479
              precision    recall  f1-score   support

           0       0.45      0.52      0.48       155
           1       0.53      0.46      0.49       185

    accuracy                           0.49       340
   macro avg       0.49      0.49      0.49       340
weighted avg       0.49      0.49      0.49       340

0.4941860465116279
0.4594594594594595
0.5345911949685535
-0.01792741475726116


In [9]:
mnb = MultinomialNB()

parameters = {
            'fit_prior': [True, False],
            'class_prior': [None, [0.5, 0.5], [0.6, 0.4], [0.4, 0.6]]
        }

mnb_gsc = GridSearchCV(mnb, parameters, n_jobs=5, 
                   cv=cv, 
                   verbose=0, refit=True)

mnb_gsc.fit(x, y)
print(mnb_gsc.best_params_, mnb_gsc.best_score_)

y_pred = mnb_gsc.predict(x_dev)
print(classification_report(y_dev, y_pred))
print(f1_score(y_dev, y_pred))
print(recall_score(y_dev, y_pred))
print(precision_score(y_dev, y_pred))
print(matthews_corrcoef(y_dev, y_pred))

{'class_prior': None, 'fit_prior': False} 0.5847154471544714
              precision    recall  f1-score   support

           0       0.43      0.56      0.49       155
           1       0.51      0.38      0.44       185

    accuracy                           0.46       340
   macro avg       0.47      0.47      0.46       340
weighted avg       0.48      0.46      0.46       340

0.43827160493827166
0.3837837837837838
0.5107913669064749
-0.0556446501177384


In [10]:
def binary_f1(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall)/(precision + recall + K.epsilon())
    
    return f1_val

x = x[:,:,None]

red_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", 
            factor=0.5,
            patience=2, 
            min_lr=0.00001,
            verbose=1)



In [11]:
model_lstm = Sequential()

model_lstm.add(LSTM(64, input_shape = x.shape[1:], return_sequences = True))
model_lstm.add(LSTM(64))

model_lstm.add(Dense(64, activation = 'relu'))
model_lstm.add(Flatten())
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics = [binary_f1])

model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 21, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 54,145
Trainable params: 54,145
Non-trainable params: 0
_________________________________________________________________


In [12]:
batch_size = 128

model_lstm.fit(x, y,
            batch_size=batch_size,
            epochs=20,
            validation_data=(x_dev, y_dev),
            class_weight=class_weights,
            shuffle=True, 
            callbacks=[red_lr]
            )


Epoch 1/20
25/25 [==============================] - 9s 87ms/step - loss: 0.6927 - binary_f1: 0.5232 - val_loss: 0.6912 - val_binary_f1: 0.6777 - lr: 0.0010
Epoch 2/20
25/25 [==============================] - 1s 32ms/step - loss: 0.6893 - binary_f1: 0.5224 - val_loss: 0.7109 - val_binary_f1: 0.6672 - lr: 0.0010
Epoch 3/20
25/25 [==============================] - ETA: 0s - loss: 0.6927 - binary_f1: 0.5941
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
25/25 [==============================] - 1s 39ms/step - loss: 0.6927 - binary_f1: 0.5941 - val_loss: 0.6914 - val_binary_f1: 0.6777 - lr: 0.0010
Epoch 4/20
25/25 [==============================] - 1s 43ms/step - loss: 0.6862 - binary_f1: 0.5729 - val_loss: 0.6967 - val_binary_f1: 0.6659 - lr: 5.0000e-04
Epoch 5/20
23/25 [==========================>...] - ETA: 0s - loss: 0.6830 - binary_f1: 0.6943
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
25/25 [==============================] - 

In [13]:
y_pred = model_lstm.predict(x_dev)
y_pred_final = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_dev, y_pred_final))
print(f1_score(y_dev, y_pred_final))
print(recall_score(y_dev, y_pred_final))
print(precision_score(y_dev, y_pred_final))
print(matthews_corrcoef(y_dev, y_pred_final))

11/11 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.51      0.13      0.21       155
           1       0.55      0.90      0.68       185

    accuracy                           0.55       340
   macro avg       0.53      0.51      0.44       340
weighted avg       0.53      0.55      0.47       340

0.6831275720164609
0.8972972972972973
0.5514950166112956
0.04115093307325052


In [14]:
model_bilstm  =  Sequential()

model_bilstm.add(Bidirectional(LSTM(64, input_shape = x.shape[1:], return_sequences=True)))
model_bilstm.add(Bidirectional(LSTM(64)))

model_bilstm.add(Dense(64, activation = 'relu'))
model_bilstm.add(Flatten())
model_bilstm.add(Dense(1, activation = 'sigmoid'))

model_bilstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics = [binary_f1])
model_bilstm.build(input_shape=x.shape)
model_bilstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (3075, 21, 128)          33792     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (3075, 128)              98816     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (3075, 64)                8256      
                                                                 
 flatten_1 (Flatten)         (3075, 64)                0         
                                                                 
 dense_3 (Dense)             (3075, 1)                 65        
                                                                 
Total params: 140,929
Trainable params: 140,929
Non-tr

In [15]:
batch_size = 128

model_bilstm.fit(x, y,
            batch_size=batch_size,
            epochs=20,
            validation_data=(x_dev, y_dev),
            class_weight=class_weights,
            shuffle=True,
            callbacks=[red_lr])

Epoch 1/20
25/25 [==============================] - 17s 168ms/step - loss: 0.6938 - binary_f1: 0.4421 - val_loss: 0.6972 - val_binary_f1: 0.0000e+00 - lr: 0.0010
Epoch 2/20
25/25 [==============================] - 1s 59ms/step - loss: 0.6920 - binary_f1: 0.2485 - val_loss: 0.6983 - val_binary_f1: 0.0000e+00 - lr: 0.0010
Epoch 3/20
25/25 [==============================] - 1s 60ms/step - loss: 0.6924 - binary_f1: 0.4211 - val_loss: 0.6896 - val_binary_f1: 0.7007 - lr: 0.0010
Epoch 4/20
25/25 [==============================] - 1s 59ms/step - loss: 0.6894 - binary_f1: 0.5143 - val_loss: 0.6914 - val_binary_f1: 0.6777 - lr: 0.0010
Epoch 5/20
24/25 [===========================>..] - ETA: 0s - loss: 0.6857 - binary_f1: 0.6019
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
25/25 [==============================] - 2s 69ms/step - loss: 0.6857 - binary_f1: 0.6098 - val_loss: 0.7060 - val_binary_f1: 0.6795 - lr: 0.0010
Epoch 6/20
25/25 [==============================] 

In [16]:
y_pred = model_bilstm.predict(x_dev)
y_pred_final = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_dev, y_pred_final))
print(f1_score(y_dev, y_pred_final))
print(recall_score(y_dev, y_pred_final))
print(precision_score(y_dev, y_pred_final))
print(matthews_corrcoef(y_dev, y_pred_final))

11/11 [==============================] - 2s 10ms/step
              precision    recall  f1-score   support

           0       0.49      0.42      0.45       155
           1       0.57      0.63      0.60       185

    accuracy                           0.54       340
   macro avg       0.53      0.53      0.52       340
weighted avg       0.53      0.54      0.53       340

0.596938775510204
0.6324324324324324
0.5652173913043478
0.05285228243785865
